In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import matplotlib.pyplot as plt
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from captum.attr import IntegratedGradients
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import torch
from collections import defaultdict
from datasets import Dataset, load_dataset, DatasetDict
print(f"Dispositivo disponible: {torch.cuda.current_device()} ({torch.cuda.get_device_name()})")

Dispositivo disponible: 0 (NVIDIA GeForce RTX 4090)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ddorado/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

file_path = 'trans_def.csv'

df = pd.read_csv(file_path)


df['about'] = df['about'].fillna('Vacia')
df['goal'] = df['goal'].fillna('Vacia')
df['description'] = df['description'].fillna('Vacia')
df['motivation'] = df['motivation'].fillna('Vacia')

def generarTextTemplate(row, opc=0):
    text = ""

    # Process 'about' if opc is 0 (all) or 1 (only about)
    if opc == 0 or opc == 1:
        if row['about'] is not None and pd.notna(row['about']):
            text += row['about'] + "."


    # Process 'description' if opc is 0 (all) or 2 (only description)
    if opc == 0 or opc == 2:
        if row['description'] is not None and pd.notna(row['description']):
            text += row['description'] + "."


    # Process 'motivation' if opc is 0 (all) or 3 (only motivation)
    if opc == 0 or opc == 3:
        if row['motivation'] is not None and pd.notna(row['motivation']):
            text += row['motivation'] + "."

    # Process 'goal' if opc is 0 (all) or 4 (only goal)
    if opc == 0 or opc == 4:
        if row['goal'] is not None and pd.notna(row['goal']):
            text += str(row['goal']) + "."


    return pd.Series({'text': text, 'label': 1 if row['status'] == 'SI' else 0})

def preparar_datasets(df, dft,w ,sample_size_train=3, sample_size_test=3, model_name='PlanTL-GOB-ES/longformer-base-4096-bne-es'):
    # Aplicar plantilla de texto
    df2 = pd.DataFrame([generarTextTemplate(df.iloc[i],w) for i in range(len(df))])
    dft2 = pd.DataFrame([generarTextTemplate(dft.iloc[i],w) for i in range(len(dft))])

# Separar el dataframe por clase
    df_clase_0 = df2[df2['label'] == 0]  # Clase minoritaria
    df_clase_1 = df2[df2['label'] == 1]  # Clase mayoritaria

# Número de ejemplos de la clase minoritaria (0)
    n_0 = len(df_clase_0)


# Queremos 3 veces más ejemplos de la clase 1 que de la clase 0
    n_1_deseado = n_0 * sample_size_train  # Número deseado de ejemplos de clase 1

# Hacer undersampling de la clase mayoritaria (1) para obtener la proporción 3:1
    df_clase_1_downsampled = df_clase_1.sample(n=n_1_deseado, replace=False, random_state=42)

# Concatenar las dos clases después de hacer el undersampling
    df_balanced = pd.concat([df_clase_0, df_clase_1_downsampled])

# Mezclar el conjunto de datos balanceado
    df2_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

    tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)
    dataset = Dataset.from_pandas(df2_balanced)
    datasett = Dataset.from_pandas(dft2)
# Verifica el límite del modelo cargado
    model_max_length = tokenizer.model_max_length

    def tokenize_function(example):
        return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=min(model_max_length, 1024),  # máximo entre lo permitido y lo deseado
        return_attention_mask=True
    )





    # Aplicar tokenización
    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    tokenized_datasett = datasett.map(tokenize_function, batched=True)

    # Convertir a tensores
    tokenized_dataset = tokenized_dataset.with_format("torch", columns=["input_ids", "attention_mask", "label"])
    tokenized_datasett = tokenized_datasett.with_format("torch", columns=["input_ids", "attention_mask", "label"])

    # Dividir datasets
    split_dataset = tokenized_dataset.train_test_split(test_size=0.3, seed=42)
    train_data = split_dataset['train']
    valid_data = split_dataset['test']
    test_data = tokenized_datasett

    return train_data, valid_data, test_data

dftr, dft = train_test_split(
    df,
    test_size=0.2,
    stratify=df['status'],
    random_state=42
)



In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from captum.attr import IntegratedGradients
from collections import defaultdict
import matplotlib.pyplot as plt
import re
import unicodedata

def corregir_codificacion(text):
    try:
        # Corrige textos mal codificados (ej: latin1 en vez de utf8)
        text = text.encode('latin1').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError):
        pass
    # Normaliza caracteres Unicode
    text = unicodedata.normalize('NFKC', text)
    return text

def visualizar_tokens_influyentes(model, tokenizer, texts, device, top_n=15, stop_words=None):
    model.eval()

    if stop_words is None:
        from nltk.corpus import stopwords
        stop_words = set(stopwords.words('spanish'))

    def procesar_texto(text):
        # Corrige codificación antes de tokenizar
        text = corregir_codificacion(text)

        inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        ref_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
        ref_input_ids = torch.full_like(input_ids, ref_token_id)
        input_embeddings = model.roberta.embeddings(input_ids)
        ref_embeddings = model.roberta.embeddings(ref_input_ids)

        def forward_func(inputs_embeds, attention_mask):
            logits = model(inputs_embeds=inputs_embeds, attention_mask=attention_mask).logits
            return torch.softmax(logits, dim=1)[:, 1]  # Probabilidad clase positiva

        ig = IntegratedGradients(forward_func)
        attributions, _ = ig.attribute(
            inputs=input_embeddings,
            baselines=ref_embeddings,
            additional_forward_args=(attention_mask,),
            return_convergence_delta=True
        )

        tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

        # Limpieza tokens: elimina prefijos de subpalabras comunes en RoBERTa (Ġ/ġ)
        tokens = [token.lstrip('Ġġ') for token in tokens]

        attributions_sum = attributions.sum(dim=-1).squeeze(0)

        # Normalizar por norma L2
        attributions_sum = attributions_sum / torch.norm(attributions_sum)

        words = []
        current_word = ""
        current_score = 0.0

        for token, score in zip(tokens, attributions_sum.tolist()):
            # Ignorar tokens especiales y tokens de formato <...>
            if token in ["<s>", "</s>", "<pad>", "<unk>",]:
                continue
            if re.fullmatch(r"<.*?>", token):
                continue

            # En RoBERTa, no se usan '##' para subpalabras, pero si tienes otros modelos verifica aquí
            if token.startswith("##"):
                current_word += token[2:]
                current_score += score
            else:
                if current_word:
                    words.append((current_word, current_score))
                current_word = token
                current_score = score
        if current_word:
            words.append((current_word, current_score))

        filtered_words = [
            (w.lower(), s) for w, s in words
            if w.lower() not in stop_words
            and len(w) > 2
            and not re.fullmatch(r"[\W_]+", w)
            and not re.fullmatch(r"\d+", w)
            and not re.fullmatch(r"\[.*?\]", w)
            and w.lower() != 'vacia'
            and w.lower() != 'cia'
        ]

        return filtered_words

    token_scores = defaultdict(float)
    for text in texts:
        words = procesar_texto(text)
        for word, score in words:
            token_scores[word] += score

    words_sorted = sorted(token_scores.items(), key=lambda x: abs(x[1]), reverse=True)[:top_n]

    return dict(token_scores)

def fusionar_y_graficar_scores(lista_scores, top_n=15, save_path=None, file_name=None):
    from collections import defaultdict
    import matplotlib.pyplot as plt
    import os

    token_scores = defaultdict(float)
    for scores in lista_scores:
        for token, score in scores.items():
            token_scores[token] += score

    positivos = [(t, s) for t, s in token_scores.items() if s > 0]
    negativos = [(t, s) for t, s in token_scores.items() if s < 0]

    top_positivos = sorted(positivos, key=lambda x: x[1], reverse=True)[:top_n]
    top_negativos = sorted(negativos, key=lambda x: x[1])[:top_n]

    words_sorted = top_positivos + top_negativos

    if not words_sorted:
        print("No hay tokens para mostrar")
        return token_scores

    labels, values = zip(*words_sorted)
    colors = ['red' if v > 0 else 'blue' for v in values]
    aligns = ['left' if v > 0 else 'right' for v in values]
    text_values = [f"+{v:.2f}" if v > 0 else f"{v:.2f}" for v in values]

    plt.figure(figsize=(10, 12))
    bars = plt.barh(range(len(labels)), values, color=colors)
    plt.yticks(range(len(labels)), labels)
    plt.axvline(0, color='black', linewidth=0.8)
    plt.xlabel("Atribución acumulada")

    for i, (bar, text_val, align) in enumerate(zip(bars, text_values, aligns)):
        x = bar.get_width()
        offset = 0.02 if x > 0 else -0.02
        plt.text(x + offset, bar.get_y() + bar.get_height() / 2, text_val,
                 va='center', ha='left' if align == 'left' else 'right',
                 color='black', fontsize=9)

    plt.title(f"Top {top_n} tokens positivos y top {top_n} negativos más influyentes (fusionados)")
    plt.tight_layout()

    if save_path and file_name:
        os.makedirs(save_path, exist_ok=True)
        full_path = os.path.join(save_path, file_name)
        plt.savefig(full_path, dpi=300, bbox_inches='tight')
        print(f"Gráfica guardada en: {full_path}")
        plt.close()
    else:
        plt.show()

    return dict(token_scores)


In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

lista_global=[]
for j in  range(1,5):
    lista_de_scores = []
    for i in range(0,3):
        model_path = f"./Rb/RbB{i}-M{j}"
        train_data, valid_data, test_data = preparar_datasets(dftr, dft, j, 2,model_name=model_path)
        model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        scores = visualizar_tokens_influyentes(model, tokenizer, test_data['text'], device)
        lista_de_scores.append(scores)
        del model
        del tokenizer

    fusionar_y_graficar_scores(lista_de_scores,save_path='./', file_name=f'Rb-{j}.png')
    lista_global=lista_global+lista_de_scores
fusionar_y_graficar_scores(lista_global,save_path='./', file_name='GlobalRb.png')

Map:   0%|          | 0/888 [00:00<?, ? examples/s]

Map:   0%|          | 0/471 [00:00<?, ? examples/s]

Map:   0%|          | 0/888 [00:00<?, ? examples/s]

Map:   0%|          | 0/471 [00:00<?, ? examples/s]

Map:   0%|          | 0/888 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/471 [00:00<?, ? examples/s]

Map:   0%|          | 0/888 [00:00<?, ? examples/s]

Map:   0%|          | 0/471 [00:00<?, ? examples/s]

Gráfica guardada en: ./Rb-4.png
Gráfica guardada en: ./GlobalRb.png


{'seguro': 1.146465643949341,
 'escuchado': 0.2252969007822685,
 'palabra': 0.6345266061252914,
 'cro': 1.0553698946605437,
 'wdfunding': 1.9557057195343077,
 'alguna': 1.8614178587013157,
 'vez': 6.755705299564852,
 'incluso': 0.2956874024530407,
 'participado': 0.3187850726535544,
 'alguno': 0.06419149783323519,
 'llevado': 0.26940590229060035,
 'cabo': 4.502936927303381,
 'contar': 3.0487638719641836,
 'quã©': 7.537105129444953,
 'financiar': 2.684381420222053,
 'dinero': 4.607644594807425,
 'reca': 0.16864320484455675,
 'ude': 0.019808640470728278,
 'gustarãńa': 1.6779598244756926,
 'compartir': 2.2245100279251346,
 'porquã©': 0.21929365675896406,
 'decidido': 2.157230650144811,
 'mãºsica': 5.661745481433172,
 'asãń': 9.46475403812019,
 'concretamente': 0.3349308857577853,
 'travã©s': 16.001300983611145,
 'teo': 6.302110813259787,
 'ijo': 0.03298849705606699,
 'manera': 15.364439395714726,
 'ser': 16.45243195340663,
 'opciã³n': 0.23345666844397783,
 'prev': 0.24746277066878974,
 'e

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from captum.attr import IntegratedGradients
from collections import defaultdict
import matplotlib.pyplot as plt
import re
import os
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords

def visualizar_tokens_influyentes(model, tokenizer, texts, device, top_n=15, stop_words=None):
    model.eval()

    if stop_words is None:
        stop_words = set(stopwords.words('spanish'))

    def procesar_texto(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        ref_token_id = tokenizer.pad_token_id
        ref_input_ids = torch.full_like(input_ids, ref_token_id)
        input_embeddings = model.roberta.embeddings(input_ids)
        ref_embeddings = model.roberta.embeddings(ref_input_ids)

        def forward_func(inputs_embeds, attention_mask):
            logits = model(inputs_embeds=inputs_embeds, attention_mask=attention_mask).logits
            return torch.softmax(logits, dim=1)[:, 1]

        ig = IntegratedGradients(forward_func)
        attributions, _ = ig.attribute(
            inputs=input_embeddings,
            baselines=ref_embeddings,
            additional_forward_args=(attention_mask,),
            return_convergence_delta=True
        )

        tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
        attributions_sum = attributions.sum(dim=-1).squeeze(0)
        attributions_sum = attributions_sum / torch.norm(attributions_sum)

        # Reconstrucción de palabras completas
        words = []
        current_word = ""
        current_score = 0.0

        for token, score in zip(tokens, attributions_sum.tolist()):
            if token in ["[CLS]", "[SEP]", "[PAD]"]:
                continue
            # Roberta usa el símbolo 'Ġ' para nuevos tokens
            if token.startswith("Ġ"):
                if current_word:
                    words.append((current_word, current_score))
                current_word = token[1:]
                current_score = score
            else:
                current_word += token
                current_score += score

        if current_word:
            words.append((current_word, current_score))

        filtered_words = [
            (w.lower(), s) for w, s in words
            if w.lower() not in stop_words
            and len(w) > 2
            and not re.fullmatch(r"[\W_]+", w)
            and not re.fullmatch(r"\d+", w)
            and not re.fullmatch(r"\[.*?\]", w)
            and not re.fullmatch(r"<.*?>", w)
            and w.lower() != 'vacia'
        ]

        return filtered_words

    token_scores = defaultdict(float)
    for text in texts:
        words = procesar_texto(text)
        for word, score in words:
            token_scores[word] += score

    return dict(token_scores)

def fusionar_y_graficar_scores(lista_scores, top_n=15, save_path=None, file_name=None):
    token_scores = defaultdict(float)
    for scores in lista_scores:
        for token, score in scores.items():
            token_scores[token] += score

    positivos = [(t, s) for t, s in token_scores.items() if s > 0]
    negativos = [(t, s) for t, s in token_scores.items() if s < 0]

    top_positivos = sorted(positivos, key=lambda x: x[1], reverse=True)[:top_n]
    top_negativos = sorted(negativos, key=lambda x: x[1])[:top_n]

    words_sorted = top_positivos + top_negativos
    if not words_sorted:
        print("No hay tokens para mostrar")
        return token_scores

    labels, values = zip(*words_sorted)
    colors = ['red' if v > 0 else 'blue' for v in values]
    aligns = ['left' if v > 0 else 'right' for v in values]
    text_values = [f"+{v:.2f}" if v > 0 else f"{v:.2f}" for v in values]

    plt.figure(figsize=(10, 12))
    bars = plt.barh(range(len(labels)), values, color=colors)
    plt.yticks(range(len(labels)), labels)
    plt.axvline(0, color='black', linewidth=0.8)
    plt.xlabel("Atribución acumulada")

    for i, (bar, text_val, align) in enumerate(zip(bars, text_values, aligns)):
        x = bar.get_width()
        offset = 0.02 if x > 0 else -0.02
        plt.text(x + offset, bar.get_y() + bar.get_height() / 2, text_val,
                 va='center', ha='left' if align == 'left' else 'right',
                 color='black', fontsize=9)

    plt.title(f"Top {top_n} tokens positivos y top {top_n} negativos más influyentes (fusionados)")
    plt.tight_layout()

    if save_path and file_name:
        os.makedirs(save_path, exist_ok=True)
        full_path = os.path.join(save_path, file_name)
        plt.savefig(full_path, dpi=300, bbox_inches='tight')
        print(f"Gráfica guardada en: {full_path}")
        plt.close()
    else:
        plt.show()

    return dict(token_scores)

# === USO (AJUSTA A TU CASO) ===

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lista_global = []

for j in range(1, 5):
    lista_de_scores = []
    for i in range(0, 3):
        model_path = f"./Rb/RbB{i}-M{j}"
        train_data, valid_data, test_data = preparar_datasets(dftr, dft, j, 2, model_name=model_path)
        model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        scores = visualizar_tokens_influyentes(model, tokenizer, test_data['text'], device)
        lista_de_scores.append(scores)

        del model
        del tokenizer

    fusionar_y_graficar_scores(lista_de_scores, save_path='./', file_name=f'Rb-{j}.png')
    lista_global += lista_de_scores

fusionar_y_graficar_scores(lista_global, save_path='./', file_name='GlobalRb.png')


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ddorado/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Map:   0%|          | 0/888 [00:00<?, ? examples/s]

Map:   0%|          | 0/471 [00:00<?, ? examples/s]

Map:   0%|          | 0/888 [00:00<?, ? examples/s]

Map:   0%|          | 0/471 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/888 [00:00<?, ? examples/s]

Map:   0%|          | 0/471 [00:00<?, ? examples/s]

Map:   0%|          | 0/888 [00:00<?, ? examples/s]

Map:   0%|          | 0/471 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
# ✅ 1. Instala las dependencias necesarias
# pip install transformers transformers-interpret

# ✅ 2. Importa las librerías
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import SequenceClassificationExplainer

# ✅ 3. Carga el modelo y el tokenizer (puede ser local o desde Hugging Face Hub)
model_path = "./Bt/BtB1-M3"  # Puedes reemplazarlo con tu propio modelo
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# ✅ 4. Crea el explicador
explainer = SequenceClassificationExplainer(model=model, tokenizer=tokenizer)

# ✅ 5. Define la lista de textos a analizar
textos=df['motivation']
labels_reales=df['status']=='SI'
resultados = []
class_name_mapping = {
    0: "negativo",
    1: "positivo"
}
explainer = SequenceClassificationExplainer(model, tokenizer)
for i, (texto, label_real) in enumerate(zip(textos, labels_reales), start=1):
    print(f"\n🔍 Texto {i}:\n{'-'*50}")

    # Crea nuevo explainer para cada texto

    word_attributions = explainer(texto[:512])

    palabras_relevantes = [word for word, score in word_attributions[:5]]
    pred_clase = explainer.predicted_class_name

    # Visualiza explicación directamente
    explainer.visualize()  # Muestra en el notebook la atribución

    # Guarda en resultados
    resultados.append({
        "Índice": i,
        "Texto (resumido)": texto[:100] + "...",
        "Palabras más relevantes": palabras_relevantes,
        "Predicción del modelo": pred_clase,
        "True Label": label_real
    })

# Tabla para análisis final
df = pd.DataFrame(resultados)
df.head()





🔍 Texto 1:
--------------------------------------------------



🔍 Texto 2:
--------------------------------------------------



🔍 Texto 3:
--------------------------------------------------



🔍 Texto 4:
--------------------------------------------------



🔍 Texto 5:
--------------------------------------------------


...


🔍 Texto 184:
--------------------------------------------------



🔍 Texto 185:
--------------------------------------------------



🔍 Texto 186:
--------------------------------------------------



🔍 Texto 187:
--------------------------------------------------


KeyboardInterrupt: 

In [ ]:
from transformers import AutoTokenizer
import pandas as pd

# Carga el tokenizer BETO en español
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

columnas = ['goal', 'motivation', 'description', 'about']

stats = {}

for col in columnas:
    token_counts = df[col].dropna().apply(lambda x: len(tokenizer.tokenize(str(x))))
    promedio = token_counts.mean()
    minimo = token_counts.min()
    maximo = token_counts.max()

    stats[col] = {
        "promedio_tokens": promedio,
        "min_tokens": minimo,
        "max_tokens": maximo
    }

stats_df = pd.DataFrame(stats).T
print(stats_df)

tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (647 > 512). Running this sequence through the model will result in indexing errors


             promedio_tokens  min_tokens  max_tokens
goal               60.181973         1.0      1113.0
motivation        222.482568         2.0      2836.0
description       327.849915        31.0      3573.0
about             327.062500         2.0      4394.0
